In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from runner import runner
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import make_scorer, mean_absolute_error

In [2]:
split_year_month_bool = True # data_holidays, preprocessor, runner, weather upload edilecek.
train_path = "new_data/train.csv"
test_path = "new_data/test.csv"
weather_path = "new_data/weather.csv"
holidays_path = "new_data/holidays.csv"
train_df,test_df = runner(train_path,test_path,weather_path,holidays_path,split_year_month_bool)

In [3]:
def apply_label_encoder(train_data,test_data,columns):
    encoders = {}
    for col in columns:
        encoder = LabelEncoder()
        train_data[col] = encoder.fit_transform(train_data[col])
        test_data[col] = encoder.fit_transform(test_data[col])
        encoders[col] = encoder
    return train_data, test_data

In [4]:
def calculate_mae(true_values, predicted_values):
    """
    Bu fonksiyon, gerçek değerler ve tahmin edilen değerler arasındaki ortalama mutlak hatayı hesaplar.
    
    Args:
    true_values (array): Gerçek değerlerin bulunduğu dizi.
    predicted_values (array): Tahmin edilen değerlerin bulunduğu dizi.
    
    Returns:
    float: Hesaplanan ortalama mutlak hata.
    """
    # Gerçek ve tahmin edilen değerler arasındaki farkların mutlak değerlerini al
    absolute_errors = np.abs(true_values - predicted_values)
    
    # Mutlak hataların ortalamasını hesapla
    mae = np.mean(absolute_errors)
    
    return mae

In [5]:
train_df.shape

(52913, 16)

In [6]:
test_df.shape

(1363, 15)

In [7]:
test_df["bildirimsiz_sum"] = None

In [8]:
test_df_not_rounded = test_df.copy()

In [9]:
cities = train_df["il"].unique()
for city in cities:
    ilceler = train_df.loc[train_df["il"] == city]["ilce"].unique()
    for ilce in ilceler:
        print(f"---------------------{city},{ilce} için işlem yapılıyor.----------------------")
        ## test_df 
        ilce_test_data = test_df.loc[(test_df["il"] == city) & (test_df["ilce"] == ilce)]
        selected_indexes = ilce_test_data.index
        ## train_df
        ilce_train_data = train_df.loc[(train_df["il"] == city) & (train_df["ilce"] == ilce)]
        ## labelencoder
        ilce_train_encoded,ilce_test_encoded = apply_label_encoder(ilce_train_data,ilce_test_data,["il","ilce"])
        test_data = ilce_train_encoded[-29:]
        train_data = ilce_train_encoded[:-29]
        X_train = train_data.drop(["bildirimsiz_sum"], axis = 1)
        y_train = train_data["bildirimsiz_sum"].tolist()
        X_test = test_data.drop(["bildirimsiz_sum"], axis = 1)
        y_test = test_data["bildirimsiz_sum"].tolist()
        ## Model
        model = xgb.XGBRegressor()
        params = {
            'max_depth': [3, 5, 7],
            'min_child_weight': [1, 3, 5],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.7, 0.8, 0.9],
            'colsample_bytree': [0.7, 0.8, 0.9],
            'n_estimators': [50, 100, 200]
        }
        tscv = TimeSeriesSplit(n_splits=5)
        
        mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
        
        random_search = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter=20, scoring=mae_scorer, cv=tscv, verbose=1, random_state=42, n_jobs=-1)
        random_search.fit(X_train, y_train)

        print("En iyi parametreler:", random_search.best_params_)
        print("En iyi skor:", random_search.best_score_)

        # Test veri seti üzerinde tahmin yap ve doğruluk skorunu hesapla
        best_model = random_search.best_estimator_
        predictions = best_model.predict(X_test)
        mae_result = calculate_mae(y_test, predictions)
        print("Test doğruluk skoru:", mae_result)

        ## Gerçek Tahmin yap.
        real_predictions = best_model.predict(ilce_test_encoded.drop(["bildirimsiz_sum"], axis = 1))
        test_df.loc[selected_indexes,"bildirimsiz_sum"] = np.round(real_predictions).astype(int)
        test_df_not_rounded.loc[selected_indexes,"bildirimsiz_sum"] = real_predictions

---------------------izmir,aliaga için işlem yapılıyor.----------------------
Fitting 5 folds for each of 20 candidates, totalling 100 fits
En iyi parametreler: {'subsample': 0.7, 'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 7, 'learning_rate': 0.01, 'colsample_bytree': 0.7}
En iyi skor: -2.5514677658185856
Test doğruluk skoru: 4.510365971203508
---------------------izmir,bayindir için işlem yapılıyor.----------------------
Fitting 5 folds for each of 20 candidates, totalling 100 fits
En iyi parametreler: {'subsample': 0.9, 'n_estimators': 100, 'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_bytree': 0.8}
En iyi skor: -2.5612160085321785
Test doğruluk skoru: 1.517379612758242
---------------------izmir,bayrakli için işlem yapılıyor.----------------------
Fitting 5 folds for each of 20 candidates, totalling 100 fits
En iyi parametreler: {'subsample': 0.9, 'n_estimators': 100, 'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_by

In [10]:
test_df['tarih'] = pd.to_datetime(test_df[['year', 'month', 'day']])
test_df_not_rounded['tarih'] = pd.to_datetime(test_df_not_rounded[['year', 'month', 'day']])

In [11]:
def create_submission_df(test_df):
    test_df['tarih'] = test_df['tarih'].dt.strftime('%Y-%m-%d')
    submission_df = pd.DataFrame()
    submission_df['unique_id'] = test_df['tarih'] + '-' + test_df['il'] + '-' + test_df['ilce']
    submission_df["bildirimsiz_sum"] = test_df["bildirimsiz_sum"].tolist()
    return submission_df

In [12]:
submission_df = create_submission_df(test_df)
submission_df2 = create_submission_df(test_df_not_rounded)

In [13]:
submission_df.to_csv("xgboost_hyperparameters_not_rounded_with_new_data.csv", index = False)

In [14]:
submission_df2.to_csv("xgboost_hyperparameters_rounded_with_new_data.csv", index = False)